In [1]:
import sys
if '../working' not in sys.path: 
    sys.path.append('../working')

!sage -preparse ../working/normalizer.sage
!mv ../working/normalizer.sage.py ../working/normalizer.py

!sage -preparse ../working/self_similar.sage
!mv ../working/self_similar.sage.py ../working/self_similar.py


In [2]:
from normalizer import *
from self_similar import *
prepare_gap_env()

In [3]:
normalizers(16, dim=2)

[
[    -x1 x0 + x1]  [x0 + x1     -x0]
[     x0      x1], [     x0      x1]
]

In [4]:
normalizers(5, dim=2, to_l_basis=True)

[
[x0 - 2*x1         0]
[       x1        x0]
]

In [5]:

def permute(space, repeat) -> str:
    """ Get all the words of letters from the given
    space of given length.

    Parameters
    ----------
    space   : Iterable container of str elements of len 1.
    repeat  : int, length of yielded words
    allow_same_neighbours : if False then all words that don't have same elements
                            on the neighbour positions will be returned and all
                            possible words otherwise.
                            Default False
    Yields
    -------
    str, result word
    """
    if repeat == 1:
        for el in space:
            yield [el]
    elif repeat < 1:
        yield []
    else:
        for prev in permute(space, repeat - 1):
            for el in space:
                yield prev + [el]


def all_words(space, max_len=-1):
    """ Get all possible words (infinite set) of elements from the given
    space.

    Parameters
    ----------
    space   : Iterable container of str elements of len 1.
    allow_same_neighbours : if False then all words that don't have same elements
                            on the neighbour positions will be returned and all
                            possible words otherwise.
                            Default False.
    max_len : maximum allowed length of returned words.
              Default None, which means that there is no maximum length

    Yields
    -------
    str, result word
    """
    i = 1
    while True:
        for el in permute(space, repeat=i):
            yield el
        i += 1

        if max_len != -1 and i > max_len:
            break


In [6]:
def build_group(group_index, deep=3):
    e = matrix(QQ, [[1, 0], [0, 1]])
    G = to_L_basis(group_index, dim=2)
    gens = G.GeneratorsOfGroup()
    gens = [matrix(QQ, el) for el in gens]
    gens = [el for el in gens if el[:2, :2] != e]

    e = matrix(QQ, [[1, 0, 0], [0, 1, 0], [0, 0, 1]])

    alpha = {}
    for seq in all_words(gens, max_len=deep): 
        
        el = e 
        for el2 in seq: 
            el *= el2 
        
        g = el[:2, :2]
        t = el[:2, 2]
        if str(g) not in alpha:
            alpha[str(g)] = t

    P = G.PointGroup()
    P = [matrix(QQ, el) for el in P.AsList()]

    for mtx in P:
        assert str(mtx) in alpha, 'couldnt build every element of point group'

    return G, P, alpha

In [7]:
G, P, alpha = build_group(5)

In [8]:
var('a b')

(a, b)

In [9]:
A = matrix(QQ, [[-1, 0], [2/3, 1/3]])
x = matrix([[a], [b]])
E = matrix(QQ, [[1, 0], [0, 1]])

A.det()

-1/3

In [10]:
A = A.inverse()

In [11]:
latex(A.inverse())

\left(\begin{array}{rr}
-1 & 0 \\
\frac{2}{3} & \frac{1}{3}
\end{array}\right)

In [12]:
for p in P: 
    assert str(A.inverse() * p * A) in alpha, 'not in normalizer'

In [13]:
t = matrix(QQ, [[2], [1/3]])
t = block_matrix([[A, t], [0, 1]])
self_similar(5, t, dim=2, gen_alphabet=True, change_basis=True)

ValueError: Bad matrix T, there is no virtual endomorphism

In [14]:
A.inverse().charpoly()

x^2 + 2/3*x - 1/3

In [15]:
A = normalizers(5, dim=2, to_l_basis=True)[0]
A

[x0 - 2*x1         0]
[       x1        x0]

In [16]:
x

[a]
[b]

In [17]:
conds = []
variables = []
coeffs = []
for i, g in enumerate(P): 
    conj = (A.inverse() * g * A).simplify_rational()
    conj = g
    condition = A.inverse() * alpha[str(g)] -  alpha[str(conj)]
    sym_res = (conj - E) * x
    print(sym_res)
    print(condition)
    print()

    conds.append(sym_res[0][0] - (condition[0][0] + var(f'n{i}')))
    cond_coeffs = {}
    for variable in conds[-1].arguments(): 
        cur_coeff = conds[-1].coefficients(variable, sparse=False)
        cond_coeffs[variable] = cur_coeff[1]
    coeffs.append(cond_coeffs)
    
    conds.append(sym_res[1][0] - (condition[1][0] + var(f'm{i}')))
    cond_coeffs = {}
    for variable in conds[-1].arguments(): 
        cur_coeff = conds[-1].coefficients(variable, sparse=False)
        cond_coeffs[variable] = cur_coeff[1]
    coeffs.append(cond_coeffs)
    
    variables.append(var(f'n{i}'))
    variables.append(var(f'm{i}'))

[0]
[0]
[0]
[0]

[-2*a]
[   a]
[0]
[0]



In [18]:
def get_constant(express): 
    while not isinstance(express, Integer) and not express.is_constant(): 
        express = express.coefficients(express.arguments()[0], sparse=False)[0]
    return express

for cof, con in zip(coeffs, conds): 
    cof['constant'] = get_constant(con)
coeffs

[{n0: -1, 'constant': 0},
 {m0: -1, 'constant': 0},
 {a: -2, n1: -1, 'constant': 0},
 {a: 1, m1: -1, 'constant': 0}]

In [19]:
A.inverse()

[       1/(x0 - 2*x1)                    0]
[-x1/((x0 - 2*x1)*x0)                 1/x0]

In [20]:
tmp = [con == 0 for con in conds]
res = solve(tmp, *variables)
print(*res[0], sep='\n')

n0 == 0
m0 == 0
n1 == -2*a
m1 == a


In [21]:
coeff_matrix = []
const_matrix = []
for cof in coeffs: 
    coeff_row = []
    for variable in [a, b] + variables: 
        coeff_row.append(cof.get(variable, 0))
    coeff_matrix.append(coeff_row)
    const_matrix.append([cof.get('constant', 0)])
    
coeff_matrix = matrix(QQ, coeff_matrix)
int_coeff_mtx = matrix(ZZ, coeff_matrix * 4)

const_matrix = matrix(QQ, const_matrix)
int_const_mtx = matrix(ZZ, const_matrix * 4)

int_coeff_mtx

[ 0  0 -4  0  0  0]
[ 0  0  0 -4  0  0]
[-8  0  0  0 -4  0]
[ 4  0  0  0  0 -4]

In [22]:
int_const_mtx

[0]
[0]
[0]
[0]

In [23]:
B, U, V = int_coeff_mtx.smith_form()

success = True
check = U * int_const_mtx
for i in range(len(B.rows())): 
    if check[i][0] % B[i][i]: 
        success = False 
        break 
success

True

In [24]:
print(B.str())

[4 0 0 0 0 0]
[0 4 0 0 0 0]
[0 0 4 0 0 0]
[0 0 0 4 0 0]


In [25]:
check

[0]
[0]
[0]
[0]

In [26]:

def get_constant(express): 
    while not isinstance(express, Integer) and not express.is_constant(): 
        express = express.coefficients(express.arguments()[0], sparse=False)[0]
    return express



In [27]:
mg = MatrixGroup(P)

In [28]:
mg.gens_dict()

{'[1 0]\n[0 1]': [1 0]
 [0 1],
 '[-1  0]\n[ 1  1]': [-1  0]
 [ 1  1]}

In [29]:
variables

[n0, m0, n1, m1]

In [32]:
G.IsSymmorphicSpaceGroup()

false

In [30]:
from itertools import product 

group_found = {}

for group_index in range(7, 8):
    print(f'========================================== group # {group_index} ======================================')
    G, P, alpha = build_group(group_index)
    found = []
    min_found = float('inf')
    group_found[group_index] = found
    
    print('point group:')
    print(*P, sep=',\n')
    
    print('normalizer:')
    print(*normalizers(group_index, dim=2, to_l_basis=True), sep=',\n')
    
    MAX_DET = 9
    
    for denom in range(1, MAX_DET):
    
        for A_template in normalizers(group_index, to_l_basis=True, dim=2):
            A_vars = A_template.arguments()
    
            for subs in product(range(MAX_DET), repeat=len(A_vars)):
                A = A_template.substitute({v: s for v, s in zip(A_vars, subs)})
                
                A = matrix(QQ, A)
                
                # A = matrix(QQ, [[-1/2, 1/2], [1/2, 1/2]])
                x = matrix([[a], [b]])
                E = matrix(QQ, [[1, 0], [0, 1]])
            
                if abs(A.det()) < 2 or abs(A.det()) > MAX_DET or abs(A.det()) > min_found: 
                    continue
                print('\n\n==========================================')
                print(f'testing matrix: \n{A}')
                print('determinant:', A.det())
                
                A = A.inverse()
                
                for p in P: 
                    p_conj = A.inverse() * p * A
                    assert str(p_conj) in alpha, f'{p_conj} not in normalizer'
                
                t = matrix(QQ, [[0], [0]])
                t = block_matrix([[A, t], [0, 1]])
                
                try: 
                    self_similar(group_index, t, dim=2, gen_alphabet=True, change_basis=True)
                    simple_virt = True 
                except ValueError as e: 
                    print(f'[!] Warning: no virtual endomorphism for affine transformation \n{t}')
                    simple_virt = False
                
                
                conds = []
                variables = []
                coeffs = []
                for i, g in enumerate(P): 
                    conj = A.inverse() * g * A
                    condition = A.inverse() * alpha[str(g)] -  alpha[str(conj)]
                    sym_res = (conj - E) * x
                
                    conds.append(sym_res[0][0] - (condition[0][0] + var(f'n{i}')))
                    cond_coeffs = {}
                    for variable in conds[-1].arguments(): 
                        cur_coeff = conds[-1].coefficients(variable, sparse=False)
                        cond_coeffs[variable] = cur_coeff[1]
                    coeffs.append(cond_coeffs)
                    
                    conds.append(sym_res[1][0] - (condition[1][0] + var(f'm{i}')))
                    cond_coeffs = {}
                    for variable in conds[-1].arguments(): 
                        cur_coeff = conds[-1].coefficients(variable, sparse=False)
                        cond_coeffs[variable] = cur_coeff[1]
                    coeffs.append(cond_coeffs)
                    
                    variables.append(var(f'n{i}'))
                    variables.append(var(f'm{i}'))
                
                for cof, con in zip(coeffs, conds):
                    cof['constant'] = get_constant(con)
                
                coeff_matrix = []
                const_matrix = []
                for cof in coeffs: 
                    coeff_row = []
                    for variable in [a, b] + variables: 
                        coeff_row.append(cof.get(variable, 0))
                    coeff_matrix.append(coeff_row)
                    const_matrix.append([cof.get('constant', 0)])
        
                coeff_matrix = matrix(QQ, coeff_matrix)
                int_coeff_mtx = matrix(ZZ, coeff_matrix * len(P))
                
                const_matrix = matrix(QQ, const_matrix)
                int_const_mtx = matrix(ZZ, const_matrix * len(P))
                
                B, U, V = int_coeff_mtx.smith_form()
                
                success = True
                check = U * int_const_mtx
                for i in range(len(B.rows())): 
                    if check[i][0] % B[i][i]: 
                        success = False 
                        break 
                print('translations found:', success)
        
                if success: 
                    found.append((abs(1 / A.det()), t, simple_virt))
                    min_found = min(min_found, abs(1 / A.det()))

========================================== group # 7 ======================================
point group:
[1 0]
[0 1],
[-1  0]
[ 0 -1],
[-1  0]
[ 0  1],
[ 1  0]
[ 0 -1]
normalizer:
[ 0 x1]
[x0  0],
[x1  0]
[ 0 x0]


testing matrix: 
[0 2]
[1 0]
determinant: -2
[!] Warning: no virtual endomorphism for affine transformation 
[  0   1|  0]
[1/2   0|  0]
[-------+---]
[  0   0|  1]
translations found: False


testing matrix: 
[0 3]
[1 0]
determinant: -3
[!] Warning: no virtual endomorphism for affine transformation 
[  0   1|  0]
[1/3   0|  0]
[-------+---]
[  0   0|  1]
translations found: False


testing matrix: 
[0 4]
[1 0]
determinant: -4
[!] Warning: no virtual endomorphism for affine transformation 
[  0   1|  0]
[1/4   0|  0]
[-------+---]
[  0   0|  1]
translations found: False


testing matrix: 
[0 5]
[1 0]
determinant: -5
[!] Warning: no virtual endomorphism for affine transformation 
[  0   1|  0]
[1/5   0|  0]
[-------+---]
[  0   0|  1]
translations found: False


testing matri

In [75]:
table_rows = []

for group_index in group_found: 
    found = sorted(group_found[group_index])

    table_rows.append([group_index, found[0][0], found[0][1], found[0][2]])

table_rows

[[
      [1/2   0|  0]      
      [  0   1|  0]      
      [-------+---]      
3, 2, [  0   0|  1], True
],
 [
      [1/2   0|  0]      
      [  0   1|  0]      
      [-------+---]      
4, 2, [  0   0|  1], True
],
 [
      [ -1   0|  0]      
      [2/3 1/3|  0]      
      [-------+---]      
5, 3, [  0   0|  1], True
],
 [
      [  0 1/2|  0]      
      [  1   0|  0]      
      [-------+---]      
6, 2, [  0   0|  1], True
],
 [
      [  1   0|  0]      
      [  0 1/2|  0]      
      [-------+---]      
7, 2, [  0   0|  1], True
],
 [
      [  0 1/3|  0]      
      [  1   0|  0]      
      [-------+---]      
8, 3, [  0   0|  1], True
],
 [
      [ -1  -2|  0]      
      [1/3   1|  0]      
      [-------+---]      
9, 3, [  0   0|  1], True
],
 [
       [-1/2  1/2|   0]      
       [ 1/2  1/2|   0]      
       [---------+----]      
10, 2, [   0    0|   1], True
],
 [
       [-1/2 -1/2|   0]      
       [-1/2  1/2|   0]      
       [---------+----]      
11, 2, [   

In [76]:
ends = table(table_rows, header_row=['#', 'index', 'phi', 'is simple'])
ends

#,index,phi,is simple
\(3\),\(2\),\(\left(\begin{array}{rr|r} \frac{1}{2} & 0 & 0 \\ 0 & 1 & 0 \\ \hline 0 & 0 & 1 \end{array}\right)\),\(\mathrm{True}\)
\(4\),\(2\),\(\left(\begin{array}{rr|r} \frac{1}{2} & 0 & 0 \\ 0 & 1 & 0 \\ \hline 0 & 0 & 1 \end{array}\right)\),\(\mathrm{True}\)
\(5\),\(3\),\(\left(\begin{array}{rr|r} -1 & 0 & 0 \\ \frac{2}{3} & \frac{1}{3} & 0 \\ \hline 0 & 0 & 1 \end{array}\right)\),\(\mathrm{True}\)
\(6\),\(2\),\(\left(\begin{array}{rr|r} 0 & \frac{1}{2} & 0 \\ 1 & 0 & 0 \\ \hline 0 & 0 & 1 \end{array}\right)\),\(\mathrm{True}\)
\(7\),\(2\),\(\left(\begin{array}{rr|r} 1 & 0 & 0 \\ 0 & \frac{1}{2} & 0 \\ \hline 0 & 0 & 1 \end{array}\right)\),\(\mathrm{True}\)
\(8\),\(3\),\(\left(\begin{array}{rr|r} 0 & \frac{1}{3} & 0 \\ 1 & 0 & 0 \\ \hline 0 & 0 & 1 \end{array}\right)\),\(\mathrm{True}\)
\(9\),\(3\),\(\left(\begin{array}{rr|r} -1 & -2 & 0 \\ \frac{1}{3} & 1 & 0 \\ \hline 0 & 0 & 1 \end{array}\right)\),\(\mathrm{True}\)
\(10\),\(2\),\(\left(\begin{array}{rr|r} -\frac{1}{2} & \frac{1}{2} & 0 \\ \frac{1}{2} & \frac{1}{2} & 0 \\ \hline 0 & 0 & 1 \end{array}\right)\),\(\mathrm{True}\)
\(11\),\(2\),\(\left(\begin{array}{rr|r} -\frac{1}{2} & -\frac{1}{2} & 0 \\ -\frac{1}{2} & \frac{1}{2} & 0 \\ \hline 0 & 0 & 1 \end{array}\right)\),\(\mathrm{True}\)
\(12\),\(9\),\(\left(\begin{array}{rr|r} -\frac{1}{3} & 0 & 0 \\ 0 & \frac{1}{3} & 0 \\ \hline 0 & 0 & 1 \end{array}\right)\),\(\mathrm{True}\)


In [45]:
group_index = 5
G, P, alpha = build_group(group_index)

G = [matrix(QQ, el) for el in G.GeneratorsOfGroup()]

print(f'Group num {group_index}')

or_G = [matrix(QQ, el) for el in gap(f'SpaceGroupOnLeftIT(2, {group_index})').GeneratorsOfGroup()]

print('Original:')
print('$$' + latex(or_G) + '$$')

print('In lattice basis:')
print('$$' + latex(G) + '$$')
print('Point Group: ')
print('$$' + latex(P) + '$$')

norm = normalizers(group_index, dim=2, to_l_basis=True)
print('Normalizer:')
latex(norm)

Group num 5
Original:
$$ \left[\left(\begin{array}{rrr}
-1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{array}\right), \left(\begin{array}{rrr}
1 & 0 & \frac{1}{2} \\
0 & 1 & \frac{1}{2} \\
0 & 0 & 1
\end{array}\right), \left(\begin{array}{rrr}
1 & 0 & 0 \\
0 & 1 & 1 \\
0 & 0 & 1
\end{array}\right)\right] $$
In lattice basis:
$$ \left[\left(\begin{array}{rrr}
-1 & 0 & 0 \\
1 & 1 & 0 \\
0 & 0 & 1
\end{array}\right), \left(\begin{array}{rrr}
1 & 0 & 1 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{array}\right), \left(\begin{array}{rrr}
1 & 0 & 0 \\
0 & 1 & 1 \\
0 & 0 & 1
\end{array}\right)\right] $$
Point Group: 
$$ \left[\left(\begin{array}{rr}
1 & 0 \\
0 & 1
\end{array}\right), \left(\begin{array}{rr}
-1 & 0 \\
1 & 1
\end{array}\right)\right] $$
Normalizer:


\left[\left(\begin{array}{rr}
x_{0} - 2 \, x_{1} & 0 \\
x_{1} & x_{0}
\end{array}\right)\right]

In [35]:
latex(ends)

\begin{tabular}{llll}
# & index & phi & is simple \\ \hline
$3$ & $2$ & $\left(\begin{array}{rr|r}
\frac{1}{2} & 0 & 0 \\
0 & 1 & 0 \\
\hline
 0 & 0 & 1
\end{array}\right)$ & $\mathrm{True}$ \\
$4$ & $2$ & $\left(\begin{array}{rr|r}
\frac{1}{2} & 0 & 0 \\
0 & 1 & 0 \\
\hline
 0 & 0 & 1
\end{array}\right)$ & $\mathrm{True}$ \\
$5$ & $3$ & $\left(\begin{array}{rr|r}
-1 & 0 & 0 \\
\frac{2}{3} & \frac{1}{3} & 0 \\
\hline
 0 & 0 & 1
\end{array}\right)$ & $\mathrm{False}$ \\
$6$ & $2$ & $\left(\begin{array}{rr|r}
0 & \frac{1}{2} & 0 \\
1 & 0 & 0 \\
\hline
 0 & 0 & 1
\end{array}\right)$ & $\mathrm{True}$ \\
$7$ & $2$ & $\left(\begin{array}{rr|r}
1 & 0 & 0 \\
0 & \frac{1}{2} & 0 \\
\hline
 0 & 0 & 1
\end{array}\right)$ & $\mathrm{True}$ \\
$8$ & $3$ & $\left(\begin{array}{rr|r}
0 & \frac{1}{3} & 0 \\
1 & 0 & 0 \\
\hline
 0 & 0 & 1
\end{array}\right)$ & $\mathrm{True}$ \\
$9$ & $3$ & $\left(\begin{array}{rr|r}
-1 & -2 & 0 \\
\frac{1}{3} & 1 & 0 \\
\hline
 0 & 0 & 1
\end{array}\right)$ & $\mathr

In [114]:
G = gap('SpaceGroupOnLeftIT(2, 5)')
G.PointGroup()

Group([ [ [ -1, 0 ], [ 0, 1 ] ] ])

In [115]:
G.TranslationBasis()

[ [ 1/2, 1/2 ], [ 0, 1 ] ]

In [108]:
A.inverse() * p * A

[-1  0]
[ 2  1]

In [105]:
group_found

{3: [(
   [1/2   0|  0]      
   [  0   1|  0]      
   [-------+---]      
2, [  0   0|  1], True
),
  (
   [  1   0|  0]      
   [  0 1/2|  0]      
   [-------+---]      
2, [  0   0|  1], True
),
  (
   [1/2   0|  0]      
   [  0   1|  0]      
   [-------+---]      
2, [  0   0|  1], True
),
  (
   [  1   0|  0]      
   [  0 1/2|  0]      
   [-------+---]      
2, [  0   0|  1], True
),
  (
   [1/2   0|  0]      
   [  0   1|  0]      
   [-------+---]      
2, [  0   0|  1], True
),
  (
   [  1   0|  0]      
   [  0 1/2|  0]      
   [-------+---]      
2, [  0   0|  1], True
),
  (
   [  1   0|  0]      
   [  0 1/2|  0]      
   [-------+---]      
2, [  0   0|  1], True
),
  (
   [1/2   0|  0]      
   [  0   1|  0]      
   [-------+---]      
2, [  0   0|  1], True
),
  (
   [1/2   0|  0]      
   [  0   1|  0]      
   [-------+---]      
2, [  0   0|  1], True
),
  (
   [  1   0|  0]      
   [  0 1/2|  0]      
   [-------+---]      
2, [  0   0|  1], True
),
  (
   

In [102]:
sorted(found)

[(
   [-1/3  2/3|   0]      
   [ 1/3  1/3|   0]      
   [---------+----]      
3, [   0    0|   1], True
),
 (
   [-1/3  2/3|   0]      
   [ 1/3  1/3|   0]      
   [---------+----]      
3, [   0    0|   1], True
),
 (
   [-1/3  2/3|   0]      
   [ 1/3  1/3|   0]      
   [---------+----]      
3, [   0    0|   1], True
),
 (
   [-1/3  2/3|   0]      
   [ 1/3  1/3|   0]      
   [---------+----]      
3, [   0    0|   1], True
),
 (
   [-1/3  2/3|   0]      
   [ 1/3  1/3|   0]      
   [---------+----]      
3, [   0    0|   1], True
),
 (
   [-1/3  2/3|   0]      
   [ 1/3  1/3|   0]      
   [---------+----]      
3, [   0    0|   1], True
),
 (
   [-1/3  2/3|   0]      
   [ 1/3  1/3|   0]      
   [---------+----]      
3, [   0    0|   1], True
),
 (
   [-1/3  2/3|   0]      
   [ 1/3  1/3|   0]      
   [---------+----]      
3, [   0    0|   1], True
),
 (
   [ 1/3  1/3|   0]      
   [-1/3  2/3|   0]      
   [---------+----]      
3, [   0    0|   1], True
),
 (
   [ 1/

In [57]:
B

[6 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 6 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 6 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 6 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 6 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 6 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 6 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 6 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 6 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 6 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 6 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 6 0 0]

In [61]:
check

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]

In [84]:
A_test.inverse()

[ 1/a  1/a]
[-1/a    0]

In [92]:
p

[-1  1]
[-1  0]

In [99]:
[matrix(QQ, el) for el in gap('SpaceGroupOnLeftIT(2, 14)').GeneratorsOfGroup()]

[
[ 0 -1  0]  [ 0 -1  0]  [1 0 1]  [1 0 0]
[ 1 -1  0]  [-1  0  0]  [0 1 0]  [0 1 1]
[ 0  0  1], [ 0  0  1], [0 0 1], [0 0 1]
]

In [104]:
[matrix(QQ, el) for el in to_L_basis(14, dim=2).PointGroup().AsList()]

[
[1 0]  [-1  1]  [ 0 -1]  [ 0 -1]  [ 1  0]  [-1  1]
[0 1], [-1  0], [ 1 -1], [-1  0], [ 1 -1], [ 0  1]
]

In [90]:
P_test = MatrixGroup(get_point_group_gens(14, dim=2))
list(P_test)

[
[1 0]  [ 0  1]  [-1 -1]  [ 0 -1]  [ 1  1]  [-1  0]
[0 1], [-1 -1], [ 1  0], [-1  0], [ 0 -1], [ 1  1]
]

In [91]:
alpha

{'[-1  1]\n[-1  0]': [0]
 [0],
 '[ 1  0]\n[ 1 -1]': [0]
 [0],
 '[ 0 -1]\n[ 1 -1]': [0]
 [1],
 '[-1  1]\n[ 0  1]': [0]
 [0],
 '[1 0]\n[0 1]': [0]
 [0],
 '[ 0 -1]\n[-1  0]': [ 0]
 [-1]}

In [85]:
A_test = matrix([[0, -1], [1, 1]])
A_test.inverse() * p * A_test

[ 1  3]
[-1 -2]

In [69]:
found

[(
[   0  1/3|   0]   
[-1/3    0|   0]   
[---------+----]   
[   0    0|   1], 9
),
 (
[1/3   0|  0]   
[  0 1/3|  0]   
[-------+---]   
[  0   0|  1], 9
),
 (
[  0 1/3|  0]   
[1/3   0|  0]   
[-------+---]   
[  0   0|  1], 9
),
 (
[-1/3    0|   0]   
[   0  1/3|   0]   
[---------+----]   
[   0    0|   1], 9
)]

In [49]:
matrix([[a, -a], [a, a]]).det()

2*a^2